In [28]:

#Working Copy  2022-02-25 0900AM
#pip install BeautifulSoup as b  & #pip install requests

from bs4 import BeautifulSoup as b
import numpy as np
import requests as r
import pandas as p
import math as m
import csv as cv
from datetime import date, timedelta, datetime

# getting search days from user
def getDates(txt):
    msg1 = 'Enter the search ' + txt + ' in YYYY-MM-DD format'
    startDate = input(msg1)
    gDate = 0 

    while gDate != 1:
        try:
            Syear, Smonth, Sday = map(int, startDate.split('-'))
            date1 = date(Syear, Smonth, Sday)
        except: 
            startDate = input(msg1)
            continue
        gDate = 1 
    return date1
date1 = getDates('startDate')
date2 = getDates('endDate')

#Get new search string for each day
base = date1
stringA = "https://www.fpds.gov/ezsearch/fpdsportal?q=+SIGNED_DATE%3A%5B"
base = date1
endt = base + timedelta(days=1)
stringB = str(base.year)+"%2F"+base.strftime("%m") + "%2F" + base.strftime("%d") + "%2C"+str(endt.year)+"%2F" + endt.strftime("%m") + "%2F" +endt.strftime("%d") + "%5D&s=FPDS.GOV&templateName=1.5.2&indexName=awardfull&x=30&start="
stringC = '0'
myURLsearch = (stringA + stringB + stringC)
myURLsearch
downloadURL = r.get(myURLsearch,headers={})
soup = b(downloadURL.text,"html.parser")
#CSS
fulltable_select  = soup.select('span.results_heading')
myLen = m.ceil(int(fulltable_select[1].find_all('b')[2].text)/30)
print(myLen)

runInt = 1
#runInt = int(runInt)
while base <= date2:
    endt = base + timedelta(days=1)
    stringB = str(base.year)+"%2F"+base.strftime("%m") + "%2F" + base.strftime("%d") + "%2C"+str(endt.year)+"%2F" + endt.strftime("%m") + "%2F" +endt.strftime("%d") + "%5D&s=FPDS.GOV&templateName=1.5.2&indexName=awardfull&x=24&y=7&start="
    stringC = int(stringC) + 30
    stringC = str(stringC)
    myURLsearch = (stringA + stringB + stringC)
    wID = runInt
    
    while wID < myLen:
        downloadURL = r.get(myURLsearch,headers={})
        soup = b(downloadURL.text,"html.parser")
        fulltable_select  = soup.select('span.results_text')
        datalist = []
        slop =[]
        for d in fulltable_select: 
            try: 
                datalist.append(d.a.contents)
            except:
                try:
                    datalist.append(d.span.contents)
                except:
                    #formerly known as slop
                    datalist.append(str(d.contents).replace("\\t","").replace("\\n","").replace("[","").replace("]","").replace("\'",""))
                    continue
        newlist = ["".join(d) if type(d) is list else d for d in datalist ]
        dlarr = np.array(newlist).reshape(30,19)
        #[",".join(d) if type(d) is list else   arr in dlarr]
        df = p.DataFrame(data= dlarr
        , columns=['AwardID','AwardType','LegalBusinessName','ContractingAgency','DateSigned','ActionObligation','ReferencedIDV','ContractingOffice','NAICS_Code','PSC_Code','EntityCity','UniqueEntityID_DUNS', 'EntityState','UniqueEntityID_SAM','Zip','UltimateParentUniqueEntityID_DUNS','UltimateParentLegalBusinessName','UltimateParentUniqueEntityID_SAM','CAGE_Code']
        )
        r2 = datetime.today().strftime("%Y%m%d%H%M%S")
        
        # Create run interval interger
        if wID < 2:
            fileName = 'fpds_ng_' + r2 + '.csv'
            df.to_csv(fileName,index = False)
            wID = wID + 1
        else:
            with open(fileName, "a", newline="") as file:
                writer = cv.writer(file,delimiter= ",",)
                writer.writerows(dlarr)
                wID = wID + 1
                print(wID)
base = base + timedelta(days=1);

92
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
3
4
5
6


KeyboardInterrupt: 

In [31]:

#Test Copy  2022-02-25 0900AM
#pip install BeautifulSoup as b  & #pip install requests

from bs4 import BeautifulSoup as b
import numpy as np
import requests as r
import pandas as p
import math as m
import csv as cv
from datetime import date, timedelta, datetime

# getting search days from user
def getDates(txt):
    msg1 = 'Enter the search ' + txt + ' in YYYY-MM-DD format'
    startDate = input(msg1)
    gDate = 0 

    while gDate != 1:
        try:
            Syear, Smonth, Sday = map(int, startDate.split('-'))
            date1 = date(Syear, Smonth, Sday)
        except: 
            startDate = input(msg1)
            continue
        gDate = 1 
    return date1
date1 = getDates('startDate')
date2 = getDates('endDate')

#Get new search string for each day
base = date1
stringA = "https://www.fpds.gov/ezsearch/fpdsportal?q=+SIGNED_DATE%3A%5B"
base = date1
endt = base + timedelta(days=1)
stringB = str(base.year)+"%2F"+base.strftime("%m") + "%2F" + base.strftime("%d") + "%2C"+str(endt.year)+"%2F" + endt.strftime("%m") + "%2F" +endt.strftime("%d") + "%5D&s=FPDS.GOV&templateName=1.5.2&indexName=awardfull&x=30&start="
stringC = '0'
myURLsearch = (stringA + stringB + stringC)
myURLsearch
downloadURL = r.get(myURLsearch,headers={})
soup = b(downloadURL.text,"html.parser")
#CSS
fulltable_select  = soup.select('span.results_heading')
myLen = m.ceil(int(fulltable_select[1].find_all('b')[2].text)/30)
print(myLen)

runInt = 1
#runInt = int(runInt)
wID = runInt
while base <= date2:
    endt = base + timedelta(days=1)
    stringB = str(base.year)+"%2F"+base.strftime("%m") + "%2F" + base.strftime("%d") + "%2C"+str(endt.year)+"%2F" + endt.strftime("%m") + "%2F" +endt.strftime("%d") + "%5D&s=FPDS.GOV&templateName=1.5.2&indexName=awardfull&x=24&y=50&start="
    stringC = int(stringC) + 30
    stringC = str(stringC)
    myURLsearch = (stringA + stringB + stringC)
    downloadURL = r.get(myURLsearch,headers={})
    soup = b(downloadURL.text,"html.parser")
    fulltable_select  = soup.select('span.results_text')
    datalist = []
    slop =[]
    for d in fulltable_select: 
        try: 
            datalist.append(d.a.contents)
        except:
            try:
                datalist.append(d.span.contents)
            except:
                #formerly known as slop
                datalist.append(str(d.contents).replace("\\t","").replace("\\n","").replace("[","").replace("]","").replace("\'",""))
                continue
    newlist = ["".join(d) if type(d) is list else d for d in datalist ]
    dlarr = np.array(newlist).reshape(30,19)
    #[",".join(d) if type(d) is list else   arr in dlarr]
    df = p.DataFrame(data= dlarr
    , columns=['AwardID','AwardType','LegalBusinessName','ContractingAgency','DateSigned','ActionObligation','ReferencedIDV','ContractingOffice','NAICS_Code','PSC_Code','EntityCity','UniqueEntityID_DUNS', 'EntityState','UniqueEntityID_SAM','Zip','UltimateParentUniqueEntityID_DUNS','UltimateParentLegalBusinessName','UltimateParentUniqueEntityID_SAM','CAGE_Code']
    )
    r2 = datetime.today().strftime("%Y%m%d%H%M%S")
    
    # Create run interval interger
    if wID < 2:
        fileName = 'fpds_ng_' + r2 + '.csv'
        df.to_csv(fileName,index = False)
        wID = wID + 1
    else:
        with open(fileName, "a", newline="") as file:
            writer = cv.writer(file,delimiter= ",",)
            writer.writerows(dlarr)
            wID = wID + 1
            print(wID)
if stringC >= myLen:
    base = base + timedelta(days=1)
    wID = 1 ;

92
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91


ValueError: cannot reshape array of size 190 into shape (30,19)

In [30]:
%whos

Variable           Type             Data/Info
---------------------------------------------
b                  type             <class 'bs4.BeautifulSoup'>
base               date             2022-01-24
builtins           module           <module 'builtins' (built-in)>
cv                 module           <module 'csv' from 'C:\\P<...>aconda3_64\\lib\\csv.py'>
d                  Tag              <span class="results_text<...>5XGW3">5XGW3</a>\n</span>
datalist           list             n=570
date               type             <class 'datetime.date'>
date1              date             2022-01-24
date2              date             2022-01-25
datetime           type             <class 'datetime.datetime'>
df                 DataFrame           AwardID            Awa<...> J1ADW6BL8MN5     5XGW3  
dlarr              ndarray          30x19: 570 elems, type `<U52`, 118560 bytes (115.78125 kb)
downloadURL        Response         <Response [200]>
endt               date             2022-01-25